In [ ]:
## Preamble
from numpy import pi, zeros, zeros_like, linspace, array, sqrt, sin, cos
import matplotlib.pyplot as plt

## Function Definitions
def drag(R,rho, C, m):
    return - (pi * R**2 * rho * C) / (2 * m)

# 4th Order Runge-Kutta method
def RK4(f, t, h, y):
    k1 = h*f(y,t)
    k2 = h*f(y+0.5*k1,t+0.5*h)
    k3 = h*f(y+0.5*k2,t+0.5*h)
    k4 = h*f(y+k3,t+h)
    y += (k1+2*k2+2*k3+k4)/6

# ODE system for projectile motion with air resistance
def f(y, t):
    D_alpha = D * y[0] * sqrt(y[0]**2 + y[2]**2)
    D_beta = D * y[2] * sqrt(y[0]**2 + y[2]**2) - g
    
    dydt = zeros_like(y)
    dydt[0] = D_alpha
    dydt[1] = y[0]
    dydt[2] = D_beta
    dydt[3] = y[2]
    
    return dydt

# Bisection method to find root of trajectory
def bisectMethod(t_points, y_points):
    a = int(len(t_points) / 2)
    b = len(t_points) - 1
    tol = 1e-6 
    
    while (b - a) > 1:
        c = (a + b) // 2
        if y_points[c] * y_points[a] > 0:
            a = c
        else:
            b = c
            
    root_time = (t_points[a] + t_points[b]) / 2
    root_x = (x_points[a] + x_points[b]) / 2
    
    return root_time, root_x

### Projectile Motion with Air Resistance
![diagram](.\diagram.JPG)

Starting with Newton's Law,

$m\dot{\vec{v}} = \sum \vec{F} = \vec{D}+\vec{G}$

$m\dot{\vec{v}} = \frac{-\pi R^2 \rho C}{2}\frac{\dot{x}(\dot{x}^2+\dot{y}^2)}{\sqrt{\dot{x}^2+\dot{y}^2}}\hat{x}-\left[\frac{\pi R^2 \rho C}{2}\frac{\dot{y}(\dot{x}^2+\dot{y}^2)}{\sqrt{\dot{x}^2+\dot{y}^2}}\hat{x}+mg\right]\hat{y}$

$\dot{\vec{v}}= \frac{-\pi R^2 \rho C}{2m}\frac{\dot{x}(\dot{x}^2+\dot{y}^2)\sqrt{\dot{x}^2+\dot{y}^2}}{\dot{x}^2+\dot{y}^2}\hat{x} - \left[\frac{\pi R^2 \rho C}{2m}\frac{\dot{y}(\dot{x}^2+\dot{y}^2)\sqrt{\dot{x}^2+\dot{y}^2}}{\dot{x}^2+\dot{y}^2}+g\right]\hat{y}=\frac{-\pi R^2 \rho C}{2m}\dot{x}\sqrt{\dot{x}^2+\dot{y}^2}\hat{x}- \left[\frac{\pi R^2 \rho C}{2m}\dot{y}\sqrt{\dot{x}^2+\dot{y}^2}+g\right]\hat{y}$

where $\vec{D}=\frac{-\pi R^2\rho C^2 |\vec{v}|^2}{2}\hat{v}=\frac{-\pi R^2\rho C^2}{2}(\dot{x}^2+\dot{y}^2)\frac{\dot{x}\hat{x}+\dot{y}\hat{y}}{\sqrt{\dot{x}^2+\dot{y}^2}}$.

Let $\vec{v}=\dot{x}\hat{x}+\dot{y}\hat{y}$ \& $\textrm{\textbf{D}}=\frac{-\pi R^2 \rho C}{2m}$,

$\ddot{x}=\textrm{\textbf{D}}\dot{x}\sqrt{\dot{x}^2+\dot{y}^2}$ & $\ddot{y}=\textrm{\textbf{D}}\dot{y}\sqrt{\dot{x}^2+\dot{y}^2}-g$

Let $\alpha = \dot{x}$ & $\beta = \dot{y} \rightarrow \dot{\alpha}=\textrm{\textbf{D}}\alpha\sqrt{\alpha^2 + \beta^2}$ & $\dot{\beta}=\textrm{\textbf{D}}\beta\sqrt{\alpha^2 + \beta^2} -g$

Thus, the projectile can be modeled through the following set of $1^{\textrm{st}}$ Order ODE: $\lbrace \alpha = \dot{x}, ~\dot{\alpha}=\textrm{\textbf{D}}\alpha\sqrt{\alpha^2 + \beta^2},~ \beta = \dot{y}, ~\dot{\beta}=\textrm{\textbf{D}}\beta\sqrt{\alpha^2 + \beta^2} -g\rbrace$


In [ ]:
# Parameters provided by Newman 8.7
m = 1           # kg, mass of the sphere
R = 0.08        # meters, radius of the sphere
rho = 1.22      # kg/m^3, density of air
v0 = 100        # m/s, initial velocity
theta = 1.667   # radians, 30 degrees, inclination angle
C = 0.47        # drag coefficient of a sphere
g = 9.81        # m/s^2, gravitational acceleration

D = drag(R, rho, C, m)

# Time and initial conditions
t0 = 0
tf = 21
N = 1000
h = (tf - t0) / N

y = zeros(4)
y[0] = v0 * cos(theta)
y[2] = v0 * sin(theta)

t_points = linspace(t0, tf, N)
x_points = []
y_points = []

# Time-stepping loop
for t in t_points:
    x_points.append(-y[1])
    y_points.append(y[3])
    RK4(f, t, h, y)

rootTime, rootX = bisectMethod(t_points, y_points)
print(f"Root found at time {rootTime} s, x-coordinate {rootX} m")

# Plotting
plt.figure()
plt.plot(x_points, y_points)
plt.scatter(rootX, 0, color='red', zorder=5)  # Plot root in red
plt.xlabel('X Position (m)')
plt.ylabel('Y Position (m)')
plt.title('Projectile Motion with Air Resistance')
plt.grid(True)
plt.show()

### The effect of the projectile's mass on the distance travelled is examined by plotting the trajectories of various mass values, compared to the case of no air resistance.

In [ ]:
plt.figure()
plt.xlabel('X Position (m)')
plt.ylabel('Y Position (m)')
plt.title('Trajectory of Projectiles with Varying Masses')
plt.grid(True)


m = 0.01
while m < 1:
    D = drag(R, rho, C, m)

    y = zeros(4)
    y[0] = v0 * cos(theta)
    y[2] = v0 * sin(theta)

    t_points = linspace(t0, tf, N)
    x_points = []
    y_points = []

    # Time-stepping loop
    for t in t_points:
        x_points.append(-y[1])
        y_points.append(y[3])
        RK4(f, t, h, y)

    rootTime, rootX = bisectMethod(t_points, y_points)
    print(f"With mass {m}, root found at time {rootTime} s, x-coordinate {rootX} m")

    # Plotting
    plt.plot(x_points, y_points, color="red")
    plt.scatter(rootX, 0, color='red', zorder=5)  # Plot root in red

    m += 0.1

while m < 15:
    D = drag(R, rho, C, m)

    y = zeros(4)
    y[0] = v0 * cos(theta)
    y[2] = v0 * sin(theta)

    t_points = linspace(t0, tf, N)
    x_points = []
    y_points = []

    # Time-stepping loop
    for t in t_points:
        x_points.append(-y[1])
        y_points.append(y[3])
        RK4(f, t, h, y)

    rootTime, rootX = bisectMethod(t_points, y_points)
    print(f"With mass {m}, root found at time {rootTime} s, x-coordinate {rootX} m")

    # Plotting
    plt.plot(x_points, y_points, color="blue")
    plt.scatter(rootX, 0, color='blue', zorder=5)  # Plot root in red

    m += 1

while m < 100:
    D = drag(R, rho, C, m)

    y = zeros(4)
    y[0] = v0 * cos(theta)
    y[2] = v0 * sin(theta)

    t_points = linspace(t0, tf, N)
    x_points = []
    y_points = []

    # Time-stepping loop
    for t in t_points:
        x_points.append(-y[1])
        y_points.append(y[3])
        RK4(f, t, h, y)

    rootTime, rootX = bisectMethod(t_points, y_points)
    print(f"With mass {m}, root found at time {rootTime} s, x-coordinate {rootX} m")

    # Plotting
    plt.plot(x_points, y_points, color="orange")
    plt.scatter(rootX, 0, color='orange', zorder=5)  # Plot root in red

    m += 10



# No Air Resistance
Range = -(2*v0**2 *sin(theta)*cos(theta))/g
print(Range)
plt.scatter(Range,0, color="green")

From the trajectory plot, we can see that the projectile's range converges to the case of no air resistance as mass increases.